In [2]:
library(MASS)

addSig = function (x) {
    if (x < .05) {return(paste(sprintf(x, fmt = '%#.3f'),'*',sep=""))}
    else {return(sprintf(x, fmt = '%#.3f'))}
}

# load data from file
data = read.csv("~/research_data/mozilla_voice_comparison/ppt_data.csv")
wpm = read.csv("~/research_data/mozilla_voice_comparison/voice_metadata.csv")

data = subset(data, voice != 'Text only')
data$rate.speed = ordered(data$rate.speed, levels = c("Much too slow","Too slow","Just right","Too fast","Much too fast"))
data$rate.speed.simple = data$rate.speed
levels(data$rate.speed.simple) = c('Very poor','poor','Just right','poor','Very poor')
data$rate.speed.simple = ordered(data$rate.speed.simple, levels = c('Very poor','poor','Just right'))

data = merge(data, wpm, by.x = 'voice', by.y = 'Voice')

m <- polr(rate.speed.simple ~ poly(WPM, 2), data = data, Hess=TRUE)
ctable <- coef(summary(m))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$odds.ratio = exp(ctable$Value)
ctable$pvalue = lapply(ctable$pvalue, addSig)
result = ctable

vars = c(
    'poly(WPM, 2)1' = '$\\mathit{Speed_{V}',
    'poly(WPM, 2)2' = '$\\mathit{Speed_{V}^{2}}',
    'Very poor|poor' = '$\\mathit{Very.poor|Poor}',
    'poor|Just right' = '$\\mathit{poor|Just.right}'
)

rank = c(
    'poly(WPM, 2)1' = 1,
    'poly(WPM, 2)2' = 2,
    'Very poor|poor' = 3,
    'poor|Just right' = 4
)

result = merge(result,vars, by.x=0, by.y=0)
result = merge(result,rank, by.x='Row.names', by.y=0)
result = result[order(result$y.y),]

include = c('y.x','odds.ratio', 'Value', 'Std. Error','pvalue')
result = result[,include]
header = c('Variable','Odds Ratio','Coef', 's.e.','p')
names(result) = header

result

,Variable,Odds Ratio,Coef,s.e.,p
,<fct>,<dbl>,<dbl>,<dbl>,<list>
1,$\mathit{Speed_{V},0.731268065,-0.3129752,2.12513037,0.883
2,$\mathit{Speed_{V}^{2}},0.007720977,-4.8638144,2.12362827,0.022*
4,$\mathit{Very.poor|Poor},0.028420970,-3.5606280,0.18857762,0.000*
3,$\mathit{poor|Just.right},0.400841971,-0.9141880,0.06894714,0.000*
